#  imports



In [ ]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision.models.detection._utils as det_utils
import torch.nn.functional as F
from torchvision import transforms as T

#important label
custom = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#loading a small set just for testing
#also installs devkits

#!mkdir -p /data/sets/nuimages  # Make the directory to store the nuImages dataset in.

#!wget https://www.nuscenes.org/data/nuimages-v1.0-mini.tgz  # Download the nuImages mini split.

#!tar -xf nuimages-v1.0-mini.tgz -C /data/sets/nuimages  # Uncompress the nuImages mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuImages.

In [ ]:
!pip install torchfunc

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from nuimages import NuImages

nuim = NuImages(version="v1.0-train", dataroot="/content/drive/MyDrive/AML_Project_2021/Metadata/")

# exploring the dataset 

In [ ]:

#nuim = NuImages(dataroot='/data/sets/nuimages', version='v1.0-mini', verbose=False, lazy=True)
nuim = NuImages(version="v1.0-train", dataroot="/content/drive/MyDrive/AML_Project_2021/Metadata/")

In [ ]:
#nuim = NuImages(version="v1.0-val", dataroot="/content/drive/MyDrive/AML_Project_2021/Metadata/")

In [ ]:
len(nuim.sample)

67279

In [ ]:
#data are stored as relational database
#each tables contains a list which has a dictionary as element
print("number of category",len(nuim.category))
print("example of one category", nuim.category[0]  )

number of category 25
example of one category {'token': '63a94dfa99bb47529567cd90d3b58384', 'name': 'animal', 'description': 'All animals, e.g. cats, rats, dogs, deer, birds.'}


In [ ]:
#TODO insert schema from drive

info needed for training

image: a PIL Image of size (H, W)

target: a dict containing the following fields

boxes (FloatTensor[N, 4]): the coordinates of the N bounding boxes in [x0, y0, x1, y1] format, ranging from 0 to W and 0 to H

labels (Int64Tensor[N]): the label for each bounding box. 0 represents always the background class.

image_id (Int64Tensor[1]): an image identifier. It should be unique between all the images in the dataset, and is used during evaluation

area (Tensor[N]): The area of the bounding box. This is used during evaluation with the COCO metric, to separate the metric scores between small, medium and large boxes.

iscrowd (UInt8Tensor[N]): instances with iscrowd=True will be ignored during evaluation.

(optionally) masks (UInt8Tensor[N, H, W]): The segmentation masks for each one of the objects

(optionally) keypoints (FloatTensor[N, K, 3]): For each one of the N objects, it contains the K keypoints in [x, y, visibility] format, defining the object. 
visibility=0 means that the keypoint is not visible. Note that for data augmentation, the notion of flipping a keypoint is dependent on the data representation, and you should probably adapt references/detection/transforms.py for your new keypoint representation

In [ ]:
#listing all tables of the dataset
nuim.table_names

['attribute',
 'calibrated_sensor',
 'category',
 'ego_pose',
 'log',
 'object_ann',
 'sample',
 'sample_data',
 'sensor',
 'surface_ann']

In [ ]:
#the sample can be indexed as a simple python list
sample_idx = 1
sample = nuim.sample[sample_idx]
sample

{'key_camera_token': '7e19c8ab6b184b4baa82a0bb4340bdad',
 'log_token': 'a16c68f6c9bd47efaec2d05206dd0637',
 'timestamp': 1527709308919956,
 'token': '0001746b484f4a0f939ff8c8b836a7e4'}

In [ ]:
#the foreign key from sample to sample data is key camera 
camera_token = sample['key_camera_token']
sample_data = nuim.get(table_name='sample_data', token= camera_token)
sample_data

{'calibrated_sensor_token': '25917959e82d56b1a896fea7574f9419',
 'ego_pose_token': '6e501ea92d6c460c9f562bc8b6d0e15d',
 'fileformat': 'jpg',
 'filename': 'samples/CAM_FRONT_RIGHT/n008-2018-05-30-15-20-59-0400__CAM_FRONT_RIGHT__1527709308919956.jpg',
 'height': 900,
 'is_key_frame': True,
 'next': '37bae0569c1844a0b04affdbba0a8362',
 'prev': 'd3d1c91e2be845bab3394afd30d1226c',
 'sample_token': '0001746b484f4a0f939ff8c8b836a7e4',
 'timestamp': 1527709308919956,
 'token': '7e19c8ab6b184b4baa82a0bb4340bdad',
 'width': 1600}

In [ ]:
"""
nuim.render_image(camera_token, annotation_type='all',
                  with_category=True, with_attributes=True, box_line_width=-1, render_scale=3)
"""

In [ ]:
object_tokens, surface_tokens = nuim.list_anns(sample['token'])

Printing object annotations:
85463f57d63748c1aa2075800eb41544 vehicle.bicycle ['cycle.without_rider']
d8de6f716a754d3aae803974f08adea5 movable_object.barrier []

Printing surface annotations:
2d83247e19c35e5bbfcbd54a40e0c244 flat.driveable_surface


In [ ]:
#bbox is [xmin, ymin, xmax, ymax]
sample_object = nuim.get(table_name="object_ann", token=object_tokens[0] )

sample_object


{'attribute_tokens': ['725f0121878f4c86909f256fd9df8569'],
 'bbox': [1496, 503, 1600, 628],
 'category_token': 'fc95c87b806f48f8a1faea2dcc2222a4',
 'mask': {'counts': 'ZVVgWjE4bGswMFdUTzNbazBNZVRPPjBLXGowSGNVTzsybjBbaTBnTmRWT2AybGgwYE1UV09gMmxoMGBNVFdPYTJraDBfTVZXT2EyaWgwYE1WV09gMmpoMD5PMDAwMU8wMDAxTzAwME8xTzJOMU8xMGJNXFdPXTFjaDBjTl5XT1wxYmgwZk5cV09aMWRoMGZOXFdPWjFoaDBiTlhXT14xaWgwYk5WV09dMWtoMGNOVVdPXTFraDBjTlVXT10xbGgwbE1WV089TmcxbGgwbE1YV09mMmhoMFpNWVdPOktpMW1oMG5NWFdPOEtqMW1oMGtNVFdPTTU9S2oxbGgwbE1UV09ONjpKbDFsaDBsTVVXT042NktQMmtoMGtNVVdPT2AwVTJdaDBqTVRXTzM+UTJdaTBPME8yTzBPMk8xTjJOMTAxTjJOMk4zTTJNM000TFleTw==',
  'size': [900, 1600]},
 'sample_data_token': '7e19c8ab6b184b4baa82a0bb4340bdad',
 'token': '85463f57d63748c1aa2075800eb41544'}

In [ ]:
category = nuim.get(table_name='category', token= sample_object['category_token'])
print(category)
print("category number", nuim.category.index(category))

{'token': 'fc95c87b806f48f8a1faea2dcc2222a4', 'name': 'vehicle.bicycle', 'description': 'Human or electric powered 2-wheeled vehicle designed to travel at lower speeds either on road surface, sidewalks or bike paths.'}
category number 14


# dataloader

In [ ]:
from torchvision import transforms

In [ ]:
class PennFudanDataset(torch.utils.data.Dataset): 
  def __init__(self , root, transforms,  version='v1.0-train', verbose=False, lazy=True ):
    # version=None, verbose=True, lazy = True, root, transforms=None, version=None, verbose=True, lazy = True
    #super.__init__()
    self.root = root
    self.nuim = NuImages(dataroot=root, version=version, verbose=verbose, lazy=lazy)
    print("dataset loaded using version:",version)
    
  def __getitem__(self, idx):
    #get sample at index idx
    sample = self.nuim.sample[idx]
    #get camera token to retrive and load image information
    camera_token = sample['key_camera_token']
    sample_data = self.nuim.get(table_name='sample_data', token= camera_token)
    image_path_rel = sample_data['filename']
    image_path_absolute = os.path.join(self.root, image_path_rel)
    img = Image.open(image_path_absolute).convert('RGB')

    #normalization like in imagenet
    img_tensor = T.ToTensor()(img)
    img = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img_tensor)
    
    object_tokens, surface_tokens = self.nuim.list_anns(sample['token'])

    classes = []
    boxes = []

    for obj_t in object_tokens:
      sample_object = self.nuim.get(table_name="object_ann", token=obj_t )
      box = sample_object['bbox'] #torch.as_tensor( , dtype=torch.float32
      #Skip the guilty box
      if (box[0] >= box[2]) or (box[1]>= box[3]) or (box[0]< 0 or box[1] <0 or box[2] < 0 or box[3] < 0):
        continue

      boxes.append(box)
      category = self.nuim.get(table_name='category', token= sample_object['category_token'])
      #class is +1 since category 0 is background
      obj_class = self.nuim.category.index(category)  + 1
      classes.append(obj_class)
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    target = {}
    target["boxes"] = boxes 
    target["labels"] = torch.as_tensor(classes, dtype=torch.int64)
    #target["masks"] = masks
    target["image_id"] = torch.tensor([idx])
    #target["area"] = area
    #target["iscrowd"] = iscrowd

    return img_tensor, target #ex img

  def __len__(self):
    return len(self.nuim.sample)



In [ ]:
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = PennFudanDataset( "/content/drive/MyDrive/AML_Project_2021/Metadata/", None, version="v1.0-train")



train_data_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn
)

val_dataset = PennFudanDataset( "/content/drive/MyDrive/AML_Project_2021/Metadata/", None, version="v1.0-val")
val_data_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn
)

'''
valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)'''

dataset loaded using version: v1.0-train
dataset loaded using version: v1.0-val


'\nvalid_data_loader = DataLoader(\n    valid_dataset,\n    batch_size=8,\n    shuffle=False,\n    num_workers=4,\n    collate_fn=collate_fn\n)'

# preparing the model

In [ ]:
#just a usefull thing
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
#load a coco pretrained model
import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
#se troppo pesante usare mobile net
#model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
for i in model.parameters():
  i.requires_grad=False

In [ ]:

if custom:
  #little hack
  model.roi_heads.box_head = torch.nn.Identity()

In [ ]:
import torchfunc

# Assuming model is loaded
print(torchfunc.sizeof(model)) #167.446.104 for resnet50fpn 

111863384


In [ ]:
num_classes = 26  # TODO: count categories of nuimages. they should be 27 + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class ResidualBlock(torch.nn.Module):
  def __init__(self, in_channels, out_channels, activation='relu'):
     super().__init__()
     #with a 3,3 kernel and 1 padding 1 stride there is the same size
     self.conv1 = torch.nn.Conv2d(in_channels= in_channels, out_channels= in_channels, kernel_size=(3,3), stride=1, padding=1 )
     self.bn1 = torch.nn.BatchNorm2d(in_channels)
     #keep the same size with kernel 1,1, but we have upsampled for more output channels
     self.conv2 = torch.nn.Conv2d(in_channels= in_channels, out_channels= out_channels, kernel_size=(1,1), stride=1, padding=0 )
     self.bn2 = torch.nn.BatchNorm2d(out_channels)
     #convolution for upsampling the residuals
     self.conv1_res = torch.nn.Conv2d(in_channels= in_channels, out_channels= out_channels, kernel_size=(1,1), stride=1, padding=0 )
     self.bn1_res = torch.nn.BatchNorm2d(out_channels)
     
  def forward(self, x):
      residual = x

      x = self.conv1(x)
      x = self.bn1(x)
      x = self.conv2(x)
      x = self.bn2(x)

      residual = self.conv1_res(residual)
      residual = self.bn1_res(residual)
      out = residual + x 
      out = F.relu(out)
      return out

In [ ]:
class BottleNeckBlock(torch.nn.Module):
  def __init__(self, in_channels, out_channels, activation='relu', middle_channel=256):
     super().__init__()
     #downsampling conv     
     self.conv3 = torch.nn.Conv2d(in_channels=in_channels, out_channels=middle_channel, kernel_size=1, padding=0, stride=1)
     #3x3 filter
     self.conv4 = torch.nn.Conv2d(in_channels=middle_channel, out_channels=middle_channel, kernel_size=(3,3), padding=1, stride=1)
     #upsampling
     self.conv5 = torch.nn.Conv2d(in_channels=middle_channel, out_channels=out_channels, kernel_size=(1,1), padding=0, stride=1)
     self.bn3 = torch.nn.BatchNorm2d(middle_channel)
     self.bn4 = torch.nn.BatchNorm2d(middle_channel)
     self.bn5 = torch.nn.BatchNorm2d(out_channels)
     
  def forward(self, x):
    
    x = self.conv3(x)
    x = self.bn3(x)
    x = self.conv4(x)
    x = self.bn4(x)
    x = self.conv5(x)
    x = self.bn5(x)

    out = F.relu(x) 
    return out   


In [ ]:
class NonLocalBlock(torch.nn.Module):
  def __init__(self, in_channels, out_channels, activation='relu', middle_channel = 512):
     super().__init__()
     self.mid = middle_channel
     self.conv_phy =  torch.nn.Conv2d(in_channels=in_channels, out_channels=middle_channel, kernel_size=1, padding=0, stride=1)
     self.conv_theta = torch.nn.Conv2d(in_channels=in_channels, out_channels=middle_channel, kernel_size=1, padding=0, stride=1)
     self.conv_g = torch.nn.Conv2d(in_channels=in_channels, out_channels=middle_channel, kernel_size=1, padding=0, stride=1)
     self.final_conv = torch.nn.Conv2d(in_channels=middle_channel, out_channels=out_channels, kernel_size=1, padding=0, stride=1)

     self.bn_phy = torch.nn.BatchNorm2d(middle_channel)
     self.bn_theta = torch.nn.BatchNorm2d(middle_channel)
     self.bn_g = torch.nn.BatchNorm2d(middle_channel)
     self.bn_fin = torch.nn.BatchNorm2d(out_channels)



  def forward(self, x):
      
      residual = x
      #we suppose the shape of x is 1024 x 1024 x H x W
      #where H and W usually are 7
      print("the shape of x is ---------------------------------------------------------------------------------:",x.shape)

      #those three shapes are now middle_channel x H x W
      theta = self.conv_theta(x)
      theta = self.bn_theta(theta)
      phy = self.conv_phy(x)
      phy = self.bn_phy(phy)
      g = self.conv_g(x)
      g = self.bn_g(g)

      middle_shape = theta.shape #psy and g have equal shapes
      #assigning new shape BatchSize x middle_channel x HW
      theta_new = theta.view(middle_shape[0], self.mid ,middle_shape[2] * middle_shape[3])
      psy_new = phy.view(middle_shape[0], self.mid ,middle_shape[2] * middle_shape[3])
      g_new = g.view(middle_shape[0], self.mid ,middle_shape[2] * middle_shape[3])

      #g and theta must change to shape Batch x HW x middle_channel
      g_new = g_new.permute(0, 2, 1)
      theta_new = theta_new.permute(0, 2, 1)

      #now the multiplication between theta: (HW x mid) and (mid x HW) results in Batch(?) HW x HW
      mat1 = torch.matmul(theta_new, psy_new)
      print("the shape of mat1 is ---------------------------------------------------------------------------------:",mat1.shape)
      mat1 = F.softmax(mat1, dim=-1)  
      # mat2 will have shape Batch x HW x mid
      mat2 = torch.matmul(mat1, g_new)

      #it must return to the middle shape mid x H x W
      #optimization from https://github.com/AlexHex7/Non-local_pytorch/blob/39ad90c91538d34e88865c9fb0ce4a844751346c/Non-Local_pytorch_0.3.1/lib/non_local_embedded_gaussian.py#L68
      mat2 = mat2.permute(0, 2, 1).contiguous() 
      print("the shape of mat2 is ---------------------------------------------------------------------------------:",mat2.shape)
      mat2 = mat2.view( middle_shape[0], middle_shape[1], middle_shape[2], middle_shape[3])

      #upsample to the output channel size and add residuals
      out = self.final_conv(mat2)
      out = self.bn_fin(out)
      out = out + residual
      out = F.relu(out)
      print("the shape of out is ---------------------------------------------------------------------------------:",out.shape)
      return out

    

In [ ]:
#custom predictor
class Custom_predictor(torch.nn.Module):
    def __init__(self,in_channels,num_classes):
        super(Custom_predictor,self).__init__()

        #fc_head 
        self.fc1 = torch.nn.Linear(12544, 1024)
        self.cls_score = torch.nn.Linear(1024, num_classes)

        #conv head, there may be a combinations of res, bottle and non local block as described in the paper
        #residual block upscales to 1024
        self.res_block = ResidualBlock(in_channels= 256, out_channels= 1024)
        #bottleneck block performs a 3x3 filter on a smaller scale
        self.Bneck = BottleNeckBlock(in_channels=1024 , out_channels=1024, middle_channel=256)
        #non local box improve foreground
        self.non_loc = NonLocalBlock(in_channels=1024, out_channels=1024, middle_channel=512)
        self.fc_conv = torch.nn.Linear(1024,num_classes*4)

        """ 
        self.cls_score = nn.Linear(in_channels, num_classes)
        self.bbox_pred = nn.Linear(in_channels, num_classes * 4)
        """

        
        
    def forward(self,x):
        print("shape of x is ", x.shape)

        x_conv = x

        #fc
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        scores = self.cls_score(x)
        
        #conv
        y = self.res_block(x_conv)        
        y = self.non_loc(y)
        y = self.Bneck(y)
        y = F.avg_pool3d(y , kernel_size=(4,3,3))
        y = y.flatten(start_dim=1)
        bbox_deltas = self.fc_conv(y)


        """
        x = x.flatten(start_dim=1)
        scores = self.cls_score(x)
        bbox_deltas = self.bbox_pred(x)
        """
        return scores, bbox_deltas

In [ ]:
if custom:
  #assigning the new custom head predictor, our main controbution
  model.roi_heads.box_predictor = Custom_predictor(in_features,num_classes)
  print("we use the double head architecture")


else :
  # replace the pre-trained head with a not trained one
  #this is the generic faster RCNN predictor
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
  print("we use the default predictor with the head not pre-trained")

we use the double head architecture


In [ ]:
for i in model.rpn.parameters():
  i.requires_grad=True
for i in model.roi_heads.parameters():
  i.requires_grad=True

In [ ]:
"""
for name,i in model.named_parameters():
  print(name,i.requires_grad)
print("number of parameters",torchfunc.sizeof(model)) 
"""

'\nfor name,i in model.named_parameters():\n  print(name,i.requires_grad)\nprint("number of parameters",torchfunc.sizeof(model)) \n'

In [ ]:
model.transform.max_size = 1600

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

num_epochs = 2

In [ ]:
#testing
itr = 0
"""
for images, targets in train_data_loader:
  print(itr)
  itr+=1
"""
num_epochs=10

In [ ]:
continue_training = True
last_itr = 8250

In [ ]:
if continue_training:
  if custom:
    model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/custom"+str(last_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
  else:
    model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/base"+str(last_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')

In [ ]:
train_dataset.nuim.version  #just to be sure

'v1.0-train'

# training

In [ ]:

loss_hist = Averager()
itr = 1
for epoch in range(num_epochs):
    break #TODO REMOVE
    loss_hist.reset()
    
    for images, targets in train_data_loader:

      #sample_data = nuim.get(table_name='sample_data', token= camera_token)
      #  print(images, targets, image_ids, x)
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        skip=False
        for t in range(len(targets)):
          if targets[t]['boxes'].shape == torch.Size([0]):
            skip=True
            break  #no box just background
        if skip or len(targets) == 0:
            continue

        
        if (continue_training and last_itr < itr ) or continue_training==False: #recover the training from the last step or train a new model
          loss_dict = model(images, targets)
          print("loss dict")
          print(loss_dict)
          print(" ")
          losses = sum(loss for loss in loss_dict.values())
          if custom:
            losses = loss_dict['loss_classifier']*2.0 + loss_dict['loss_box_reg'] * 2.5 + loss_dict['loss_rpn_box_reg']
          loss_value = losses.item()

          print("loss value")
          print(loss_value)
          print(" ")

          print("iteration", itr)

          loss_hist.send(loss_value)

          optimizer.zero_grad()
          losses.backward()
          optimizer.step()

          if itr % 50 == 0:
              print(f"Iteration #{itr} loss: {loss_value}")
          if itr % 250 == 0:
            print("sto salvando il modello")
            if custom:
              torch.save(model.state_dict(), "/content/drive/MyDrive/AML_Project_2021/Modelli/custom"+str(itr)+".pth")
            else:
              torch.save(model.state_dict(), "/content/drive/MyDrive/AML_Project_2021/Modelli/base"+str(itr)+".pth")
        itr += 1
        print("iterazione itr", itr)
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")   

In [ ]:
"""
#unlycky case where there is no box but just background mask
images_b =[]; targets_b = []
for t in range(len(targets)):
  if targets[t]['boxes'].shape == torch.Size([0]):
    continue  #no box just background
  images_b.append(images[t])
  targets_b.append(targets[t])

if len(targets_b) == 0:
  continue 

"""    

In [ ]:

!nvidia-smi

In [ ]:
#!tar -tvf /content/drive/MyDrive/AML_Project_2021/nuimages-v1.0-all-samples.tgz

In [ ]:
#one time only
#!tar -xvf /content/drive/MyDrive/AML_Project_2021/nuimages-v1.0-all-samples.tgz samples/CAM_FRONT_LEFT/ -C /content/drive/MyDrive/AML_Project_2021/Data/

In [ ]:
#!mv samples/CAM_FRONT_LEFT /content/drive/MyDrive/AML_Project_2021/Metadata/samples/

In [ ]:
#!find /content/drive/MyDrive/AML_Project_2021/Metadata/samples/ -type f | wc -l

# codice per la raccolta del loss sui modelli

In [ ]:
#the idea is to load the different stored models and calculate their loss on 100 images and their accuracy and loss on 100 validation images

In [ ]:
import json

In [ ]:
#loss on 100 immages
#first of all get the model 
current_model = 250
last_model_itr = 10000
dizionario_loss = {}
with torch.no_grad():

  while (current_model < last_model_itr):

    #break #TO DO REMOVE WHEN NEED ----------------------------------------------------------------------------------------------------------------------------------------

    if custom:
      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/custom"+str(current_model)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
    else:
      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/base"+str(current_model)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
    loss = 0
    i = 0
    
    for images, targets in train_data_loader:  

      #same steps as in training for cleaning the images
      images = list(image.to(device) for image in images)
      targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
      skip=False
      for t in range(len(targets)):
        if targets[t]['boxes'].shape == torch.Size([0]):
          skip=True
          break  #no box just background
      if skip or len(targets) == 0:
          continue
      loss_dict = model(images, targets)
      print("loss dict")
      print(loss_dict)
      print(" ")
      losses = sum(loss for loss in loss_dict.values())
      if custom:
        losses = loss_dict['loss_classifier']*2.0 + loss_dict['loss_box_reg'] * 2.5 + loss_dict['loss_rpn_box_reg']
      loss_value = losses.item()

      print("loss value")
      print(loss_value)
      print(" ")
      print(" ")
      print("iterazione ",i)
      print(" ")
      loss += loss_value
      i+=1
      if (i > 100):
        break
    #loop esterno
    loss_fin = loss/100
    dizionario_loss[current_model]=loss_fin
    with open('/content/drive/MyDrive/AML_Project_2021/Modelli/loss'+str(custom)+'.json', 'w') as fp:
      json.dump(dizionario_loss, fp)
    current_model += 250
    loss = 0
    print("ho salvato loss di ", loss_fin, " del modello numero ", current_model-250)
    




  

# evaluation


Reload the model. Please instantiate the model first

In [ ]:
if custom:
  model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/custom"+str(3500)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
else:
  model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/base"+str(3500)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')

Creates the dataframe on evaluation. Initialize valid_data_loader first

In [ ]:
import pandas as pd


In [ ]:
flag = True # Set to true to stop early
max_cnt = 10           # Stops after max_cnt iteration if flag = True
cnt = 0;

#Create df
new_df = pd.DataFrame(columns=["boxes", "labels", "scores", "target_boxes", "target_labels"])

#Iterate
model.eval()
for images, targets in train_data_loader:


  images = list(img.to(device) for img in images)
  targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

  boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
  sample = images[0].permute(1,2,0).cpu().numpy()
  
  cpu_device = torch.device("cpu")

  
  
  outputs = model(images)
  print("iteration :",cnt)
  outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
  
  new_df = new_df.append({"boxes" : outputs[0]["boxes"].detach().numpy(), "labels" : outputs[0]["labels"], "scores": outputs[0]["scores"].detach().numpy(), #there was [outputs[0]["boxes"]]
                          "target_boxes" : torch.tensor(boxes), "target_labels": targets[0]["labels"],"img_idx" : cnt},  #okkio
                         ignore_index=True)

  cnt+=1
  if flag:
    if cnt==max_cnt:
      break
  




Printing object annotations:

Printing surface annotations:
a6351bb648b254f6a2e5afbb2fc07de6 flat.driveable_surface


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


shape of x is  torch.Size([1000, 256, 7, 7])
the shape of x is ---------------------------------------------------------------------------------: torch.Size([1000, 1024, 7, 7])
the shape of mat1 is ---------------------------------------------------------------------------------: torch.Size([1000, 49, 49])
the shape of mat2 is ---------------------------------------------------------------------------------: torch.Size([1000, 512, 49])
the shape of out is ---------------------------------------------------------------------------------: torch.Size([1000, 1024, 7, 7])
iteration : 0
Printing object annotations:
85463f57d63748c1aa2075800eb41544 vehicle.bicycle ['cycle.without_rider']
d8de6f716a754d3aae803974f08adea5 movable_object.barrier []

Printing surface annotations:
2d83247e19c35e5bbfcbd54a40e0c244 flat.driveable_surface
shape of x is  torch.Size([1000, 256, 7, 7])
the shape of x is ---------------------------------------------------------------------------------: torch.Size([1000, 

Save df; WIP since it transforms tensors and arrays to Strings

In [ ]:
#new_df.to_csv("/content/drive/MyDrive/def_AML.csv",index= False)

Reload the df (don't)

In [ ]:
#new_df = pd.read_csv("/content/drive/MyDrive/prova_AML.csv") #but then tensor become string...

Metrics for boxes

In [ ]:
import numpy as np
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Utility functions

In [ ]:
def get_model_scores(pred_boxes):
    """Creates a dictionary of from model_scores to image ids.
    Args:
        pred_boxes (dict): dict of dicts of 'boxes' and 'scores'
        dizionario fatto: {scores: [], boxes: []}
        dizionario fatto: img_id : {scores: [], boxes: []}
    Returns:
        dict: keys are model_scores and values are image ids (usually filenames)
    """
    model_score={}
    for img_id, val in pred_boxes.items():
        #print("---------",val)
        for score in val['scores']:
            if score not in model_score.keys():
                model_score[score]=[img_id]
            else:
                model_score[score].append(img_id)
    return model_score

In [ ]:
#input per get_model_Scores
def obtain_dict_of_dict(dd):
  new_dict = {}
  for x in range(len(dd["boxes"])):
    value_b = []
    value_s = []
    for y in dd["boxes"].keys():
      if y == x:
        value_b = dd["boxes"][y]
    for z in dd["scores"].keys():
      if z == x:
        value_s = dd["scores"][z]
    new_dict[x] = {"boxes" : value_b, "scores" : value_s}
  return(new_dict)

In [ ]:
#input dopo forse
def obtain_dict_from_img(new_df, id):
  new_dict = {"boxes" :  new_df.loc[id, "boxes"], "scores" :  new_df.loc[id, "scores"]}
  return(new_dict)

In [ ]:
#! IMPORTANT
def dict_numbox_box(df, idx):
  newd={}
  actual = df.loc[idx, "boxes"]
  #print(actual)
  #for x in actual:
  for i,y in enumerate(actual):
        #print(i, torch.tensor(y))
        newd[torch.tensor(y)]=i
  return(newd)

In [ ]:
def calc_iou( gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and 
    return the IoU ratio
    '''
    #print(pred_bbox)
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt= gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p= pred_bbox
  
    
    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt> y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p> y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct",x_topleft_p, x_bottomright_p,y_topleft_p,y_bottomright_gt)
        
         
    #if the GT bbox and predcited BBox do not overlap then iou=0
    if(x_bottomright_gt< x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox
        
        return 0.0
    if(y_bottomright_gt< y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox
        
        return 0.0
    if(x_topleft_gt> x_bottomright_p): # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox
        
        return 0.0
    if(y_topleft_gt> y_bottomright_p): # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox
        
        return 0.0
    
    
    GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_bottomright_gt -y_topleft_gt + 1)
    Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_bottomright_p -y_topleft_p + 1)
    
    x_top_left =np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])
    
    intersection_area = (x_bottom_right- x_top_left + 1) * (y_bottom_right-y_top_left  + 1)
    
    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)
   
    return intersection_area/union_area

In [ ]:
def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.
    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.
    Returns:
        dict: true positives (int), false positives (int), false negatives (int)
    """
    all_pred_indices= range(len(pred_boxes))
    all_gt_indices=range(len(gt_boxes))
    if len(all_pred_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    if len(all_gt_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    
    gt_idx_thr=[]
    pred_idx_thr=[]
    ious=[]
    
    #it's enumerating the keys of dict (?)
    for ipb, pred_box in enumerate(pred_boxes):
        #print("-------------",ipb, pred_box)
        
        for igp, gt_box in enumerate(gt_boxes):#for igp, gt_box in enumerate(gt_boxes):
            #print(igb, pred_box)
            #print("------------------------",pred_box)
            iou= calc_iou(list(gt_box), pred_box)
            
            if iou >iou_thr:
                gt_idx_thr.append( igp)#igb)
                pred_idx_thr.append(ipb) #ipb)
                ious.append(iou)
          
    iou_sort = np.argsort(ious)[::1]
    if len(iou_sort)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    else:
        gt_match_idx=[]
        pred_match_idx=[]
        for idx in iou_sort:
            gt_idx=gt_idx_thr[idx]
            pr_idx= pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if(gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp= len(gt_match_idx)
        fp= len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)
    return {'true_positive': tp, 'false_positive': fp, 'false_negative': fn}

Just to test the get_single_image_result

In [ ]:
for x in range(len(new_df)):
      gt_boxes = new_df.loc[x, "target_boxes"]#nd array, kinda ok
      pred_boxes = dict_numbox_box(new_df, x)
      print("img result", get_single_image_results(gt_boxes, pred_boxes, 0.5))
      

img result {'true_positive': 0, 'false_positive': 0, 'false_negative': 0}
img result {'true_positive': 2, 'false_positive': 64, 'false_negative': 0}
img result {'true_positive': 2, 'false_positive': 30, 'false_negative': 0}
img result {'true_positive': 0, 'false_positive': 0, 'false_negative': 0}
img result {'true_positive': 11, 'false_positive': 68, 'false_negative': 3}
img result {'true_positive': 4, 'false_positive': 33, 'false_negative': 1}
img result {'true_positive': 3, 'false_positive': 25, 'false_negative': 0}
img result {'true_positive': 3, 'false_positive': 17, 'false_negative': 0}
img result {'true_positive': 2, 'false_positive': 36, 'false_negative': 2}
img result {'true_positive': 4, 'false_positive': 24, 'false_negative': 0}


In [ ]:
def calc_precision_recall(image_results):
    """Calculates precision and recall from the set of images
    Args:
        img_results (dict): dictionary formatted like:
            {
                'img_id1': {'true_pos': int, 'false_pos': int, 'false_neg': int},
                'img_id2': ...
                ...
            }
    Returns:
        tuple: of floats of (precision, recall)
    """
    true_positive=0
    false_positive=0
    false_negative=0
    for img_id, res in image_results.items():
        true_positive +=res['true_positive']
        false_positive += res['false_positive']
        false_negative += res['false_negative']
        try:
            precision = true_positive/(true_positive+ false_positive)
        except ZeroDivisionError:
            precision=0.0
        try:
            recall = true_positive/(true_positive + false_negative)
        except ZeroDivisionError:
            recall=0.0
    return (precision, recall)

Create a dict with all the images TP, FP etc.

In [ ]:
img_dict = {}
for x in range(len(new_df)):
      gt_boxes = new_df.loc[x, "target_boxes"]#nd array, kinda ok
      pred_boxes = dict_numbox_box(new_df, x)
      
      img_dict[x] = get_single_image_results(gt_boxes, pred_boxes, 0.5) #<---- modify threshold here as desired

Precision and recall over all the images

In [ ]:
print(calc_precision_recall(img_dict))

(0.09451219512195122, 0.8378378378378378)


In [ ]:
def  get_avg_precision_at_iou(gt_boxes, pred_bb, iou_thr=0.5):
    
    model_scores = get_model_scores(pred_bb)
    sorted_model_scores= sorted(model_scores.keys())# Sort the predicted boxes in descending order (lowest scoring boxes first):
    for img_id in pred_bb.keys():
        #print(img_id)
        
        arg_sort = np.argsort(pred_bb[img_id]['scores'])
        #print( np.array(pred_bb[img_id]['boxes'])[arg_sort])
        pred_bb[img_id]['scores'] = np.array(pred_bb[img_id]['scores'])[arg_sort].tolist()
        pred_bb[img_id]['boxes'] = np.array(pred_bb[img_id]['boxes'])[arg_sort].tolist()
        pred_boxes_pruned = deepcopy(pred_bb)
    
    precisions = []
    recalls = []
    model_thrs = []
    img_results = {}# Loop over model score thresholds and calculate precision, recall
    for ithr, model_score_thr in enumerate(sorted_model_scores[:-1]):
            # On first iteration, define img_results for the first time:
        print("Model score : ", model_score_thr)
        img_ids = gt_boxes.keys() if ithr == 0 else model_scores[model_score_thr] 
        for img_id in img_ids:
               
            gt_boxes_img = gt_boxes[img_id]
            box_scores = pred_boxes_pruned[img_id]['scores']
            start_idx = 0
            for score in box_scores:
                if score <= model_score_thr:
                    pred_boxes_pruned[img_id]
                    start_idx += 1
                else:
                    break 
            # Remove boxes, scores of lower than threshold scores:
            pred_boxes_pruned[img_id]['scores']= pred_boxes_pruned[img_id]['scores'][start_idx:]
            pred_boxes_pruned[img_id]['boxes']= pred_boxes_pruned[img_id]['boxes'][start_idx:]# Recalculate image results for this image
            print(img_id)
            img_results[img_id] = get_single_image_results(gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], iou_thr=0.5)# calculate precision and recall
        prec, rec = calc_precision_recall(img_results)
        precisions = np.append(precisions, prec) #precisions.append(prec)
        recalls.append(rec)
        model_thrs.append(model_score_thr)
        precisions = np.array(precisions)
    recalls = np.array(recalls)
    prec_at_rec = []
    for recall_level in np.linspace(0.0, 1.0, 11):
        try:
            args= np.argwhere(recalls>recall_level).flatten()
            prec= max(precisions[args])
            print(recalls,"Recall")
            print(      recall_level,"Recall Level")
            print(       args, "Args")
            print(       prec, "precision")
        except ValueError:
            prec=0.0
        prec_at_rec.append(prec)
    avg_prec = np.mean(prec_at_rec) 
    return {
        'avg_prec': avg_prec,
        'precisions': precisions,
        'recalls': recalls,
        'model_thrs': model_thrs}

In [ ]:
results = get_avg_precision_at_iou(new_df["target_boxes"],obtain_dict_of_dict(new_df[["boxes", "scores", "img_idx"]]), 0.5) #<----- Modify threshold here as desired


In [ ]:
results

In [ ]:
"""
#TODO TEST
model.eval()
max_cnt = 100
model_itr = 250
max_model_itr = 8250
avg_scores = {}
with torch.no_grad():

  while(model_itr < max_model_itr):
    
    if custom:
      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/custom"+str(model_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
    else:
      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/base"+str(model_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
    

    #Create df
    new_df = pd.DataFrame(columns=["boxes", "labels", "scores", "target_boxes", "target_labels"])

    #Iterate
    cnt = 0
    for images, targets in train_data_loader:


      images = list(img.to(device) for img in images)
      targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

      boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
        
      
      outputs = model(images)
      print("iteration :",cnt)
      outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
      
      new_df = new_df.append({"boxes" : outputs[0]["boxes"].detach().numpy(), "labels" : outputs[0]["labels"], "scores": outputs[0]["scores"].detach().numpy(), #there was [outputs[0]["boxes"]]
                              "target_boxes" : torch.tensor(boxes), "target_labels": targets[0]["labels"],"img_idx" : cnt},  #okkio
                            ignore_index=True)

      cnt+=1
      if flag:
        if cnt==max_cnt:
          break
    results = get_avg_precision_at_iou(new_df["target_boxes"],obtain_dict_of_dict(new_df[["boxes", "scores", "img_idx"]]), 0.5)
    avg_scores[model_itr] = results['avg_prec']
    with open('/content/drive/MyDrive/AML_Project_2021/Modelli/acc_val'+str(custom)+'.json', 'w') as fp:
      json.dump(avg_scores, fp)
    model_itr += 250
"""
    

'\n#TODO TEST\nmodel.eval()\nmax_cnt = 100\nmodel_itr = 250\nmax_model_itr = 8250\navg_scores = {}\nwith torch.no_grad():\n\n  while(model_itr < max_model_itr):\n    \n    if custom:\n      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/custom"+str(model_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device(\'cpu\')\n    else:\n      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/base"+str(model_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device(\'cpu\')\n    \n\n    #Create df\n    new_df = pd.DataFrame(columns=["boxes", "labels", "scores", "target_boxes", "target_labels"])\n\n    #Iterate\n    cnt = 0\n    for images, targets in train_data_loader:\n\n\n      images = list(img.to(device) for img in images)\n      targets = [{k: v.to(device) for k, v in t.items()} for t in targets]\n\n      boxes = targets[0][\'boxes\'].cpu().numpy().astype(np.int32)\n        \n      \n      outputs = m

# evaluation multipla

In [ ]:
import json 

In [ ]:
custom

True

In [ ]:
# evaluation multipla



model.eval()
max_cnt = 250
model_itr = 4500
max_model_itr = 10000
avg_scores = {}
with torch.no_grad():

  while(model_itr <= max_model_itr):
    
    if custom:
      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/custom"+str(model_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
    else:
      model.load_state_dict( torch.load( "/content/drive/MyDrive/AML_Project_2021/Modelli/base"+str(model_itr)+".pth")) #if on CPU, add this: CPU map_location=torch.device('cpu')
    

    #Create df
    new_df = pd.DataFrame(columns=["boxes", "labels", "scores", "target_boxes", "target_labels"])

    #Iterate
    cnt = 0
    for images, targets in val_data_loader:


      images = list(img.to(device) for img in images)
      targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

      boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
        
      
      outputs = model(images)
      print("iteration :",cnt)
      outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
      
      new_df = new_df.append({"boxes" : outputs[0]["boxes"].detach().numpy(), "labels" : outputs[0]["labels"], "scores": outputs[0]["scores"].detach().numpy(), #there was [outputs[0]["boxes"]]
                              "target_boxes" : torch.tensor(boxes), "target_labels": targets[0]["labels"],"img_idx" : cnt},  #okkio
                            ignore_index=True)

      cnt+=1
      if flag:
        if cnt==max_cnt:
          break
    results = get_avg_precision_at_iou(new_df["target_boxes"],obtain_dict_of_dict(new_df[["boxes", "scores", "img_idx"]]), 0.5)
    avg_scores[model_itr] = results['avg_prec']
    with open('/content/drive/MyDrive/AML_Project_2021/Modelli/acc_val'+str(custom)+'.json', 'w') as fp:
      json.dump(avg_scores, fp)
    print("dizionario acc",avg_scores )
    model_itr += 250

    